## This file contains: use ResNet18 to find the "best" augmentation method. And get the accuracy results including No augmentation, augmentation1, augmentation2, augmentation3, augmentation4

In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
path = "/content/drive/MyDrive/EE541_project/"

In [2]:
import os
import math
import glob
import time
import copy
import h5py
import numpy as np
import torch
import torchvision
from torchvision import transforms
import torch.optim as optim
import torch.nn as nn
from skimage.util import random_noise
from torch.utils.data import random_split, Dataset, DataLoader

In [3]:
device = (torch.device('cuda') if torch.cuda.is_available()
                  else torch.device('cpu'))
print(f"Training on device {device}.")

data_hdf5_root = os.path.join(path, "data/asl/data.hdf5")

Training on device cuda.


In [4]:
random_seed = 123 # set random seed for reproductivity
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)

#### HDF5Dataset Class and Augmentation Noise

In [5]:
class HDF5Dataset(Dataset):
    def __init__(self, file_path, name, transform=None):
        super().__init__
        self.file_path = file_path
        self.data_cache = {}
        self.transform = transform
        self.name = str(name)
        self.name_label = str(name) + "_label"
        self.size = None
        with h5py.File(file_path, 'r') as hf:
            self.data_cache[self.name] = hf[self.name][:]
            self.data_cache[self.name_label] = hf[self.name_label][:]
            self.size = len(hf[self.name_label])

    def __getitem__(self, index):
        img = self.data_cache[self.name][index]
        if self.transform:
            img = self.transform(img)
        else:
            img = torch.from_numpy(img)

        label = self.data_cache[self.name_label][index]
        return img, label

    def __len__(self):
        return self.size

In [6]:
class gaussian_noise():
    def __init__(self, mean, stddev):
       self.mean = mean
       self.stddev = stddev
    def __call__(self, img):
       gauss_img = random_noise(img, mode='gaussian', mean=self.mean, var=self.stddev, clip=True)
       return torch.tensor(gauss_img, dtype=torch.float32)
       
class salt_pepper_noise():
    def __init__(self, amount):
      self.amount = amount
    def __call__(self, img):
       sp_img = random_noise(img, mode='s&p', amount=self.amount)
       return torch.tensor(sp_img, dtype=torch.float32)

#### Training loop

In [ ]:
def train_loop(n_epochs, train_loader, val_loader, model, loss_fn, optimizer, model_path):
    loss_train_list = []
    acc_train_list = []
    loss_val_list = []
    acc_val_list = []
    acc_val_max = 0
    n_stop = 0
    for epoch in range(1, n_epochs+1):
        model.train()
        loss_train = 0.0
        correct = 0
        total = 0
        start_time = time.time()
        print(f"Epoch {epoch} Training start:")
        for batch_i, (imgs, labels) in enumerate(train_loader):
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)

            loss = loss_fn(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _, predicted = torch.max(outputs, dim=1)
            total += labels.shape[0]
            correct += int((predicted == labels).sum())
            loss_train += loss.item()
            if (batch_i) % 200 == 0:
                print ('Epoch {}, Step {}, Loss: {:.4f}'
                .format(epoch, batch_i, loss.item()))
                #torch.save(model.state_dict(), path_store_net)
        
        loss_train_list.append(loss_train/len(train_loader))
        acc_train_list.append(100 * correct/total)
        end_time = time.time()
        print('Duration: {:2.2f} minutes, Epoch {}, Training loss {:.4f}, Training accuracy {:2.3f}%'
        .format((end_time - start_time)/60 ,epoch,
            loss_train/len(train_loader), 100*correct/total))

        # validation
        loss_val = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            model.eval()
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)
                total += labels.shape[0]
                correct += int((predicted == labels).sum())
                loss = loss_fn(outputs, labels)
                loss_val += loss.item()

            acc_val = 100*correct/total
            loss_val_list.append(loss_val/len(val_loader))#loss per batch
            acc_val_list.append(acc_val)
            print('Epoch {}, Validation loss {:.4f}, Validation accuracy {:2.3f}%'
            .format(epoch, loss_val/len(val_loader), acc_val))
        
            if acc_val > acc_val_max:
                acc_val_max = acc_val
                model_scripted = torch.jit.script(model)
                model_scripted.save(model_path)
                print('Detect Improvement, Save Model')
            else:
                n_stop += 1
    
        if(n_stop == 2):
            break
    return (loss_train_list, acc_train_list, loss_val_list, acc_val_list)

#### The 1st augmentation

In [7]:
train_transforms = transforms.Compose([transforms.ToTensor(),
                                       transforms.Normalize((0.5190, 0.4993, 0.5141),
                                                            (0.2280, 0.2555, 0.2637)),
                                       transforms.Resize(224),
                                       transforms.RandomHorizontalFlip(p=0.5),
                                       transforms.RandomApply(transforms=[gaussian_noise(0.1, 0.3)], p=0.3),
                                       transforms.RandomApply(transforms=[salt_pepper_noise(0.3)], p=0.3),
                                       transforms.RandomRotation((0, 5))])

test_transforms = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize((0.5190, 0.4993, 0.5141),
                                                         (0.2280, 0.2555, 0.2637)),
                                    transforms.Resize(224)])

In [ ]:
train_dataset = HDF5Dataset(data_hdf5_root, name = "train", transform = train_transforms)
val_dataset = HDF5Dataset(data_hdf5_root, name = "val", transform = test_transforms)

print(f"train size is {len(train_dataset)}")
print(f"validation size is {len(val_dataset)}")
print(f"All data size is {len(train_dataset)+len(val_dataset)}")

In [ ]:
batch_size = 128
n_epochs = 10

model = torchvision.models.resnet18(pretrained=True)
input_num_fc = model.fc.in_features
model.fc = nn.Linear(input_num_fc, 29, bias=True)
model = model.to(device=device)

optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum=0.9)

loss_fn = nn.CrossEntropyLoss()

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

model_path = os.path.join(path, "data/asl/aug_models/ResNet18_aug1.pt")
res = train_loop(n_epochs, train_loader, val_loader, model, loss_fn, optimizer, model_path)

#### The 2nd Augmentation

In [ ]:
train_transforms = transforms.Compose([transforms.ToTensor(),
                                       transforms.Normalize((0.5190, 0.4993, 0.5141),
                                                         (0.2280, 0.2555, 0.2637)),
                                       transforms.Resize(224),
                                       transforms.RandomHorizontalFlip(p=0.5),
                                       transforms.RandomApply(transforms=[gaussian_noise(0, 0.35)], p=0.3),
                                       transforms.RandomApply(transforms=[salt_pepper_noise(0.35)], p=0.3),
                                       transforms.RandomRotation((0, 10))])

train_dataset = HDF5Dataset(data_hdf5_root, name = "train", transform = train_transforms)

In [ ]:
batch_size = 128
n_epochs = 10

model = torchvision.models.resnet18(pretrained=True)
input_num_fc = model.fc.in_features
model.fc = nn.Linear(input_num_fc, 29, bias=True)
model = model.to(device=device)

optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum=0.9)

loss_fn = nn.CrossEntropyLoss()

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

model_path = os.path.join(path, "data/asl/aug_models/ResNet18_aug2.pt")
res = train_loop(n_epochs, train_loader, val_loader, model, loss_fn, optimizer, model_path)

#### The 3rd Augmentation

In [ ]:
train_transforms = transforms.Compose([transforms.ToTensor(),
                                       transforms.Normalize((0.5190, 0.4993, 0.5141),
                                                         (0.2280, 0.2555, 0.2637)),
                                       transforms.Resize(224),
                                       transforms.RandomHorizontalFlip(p=0.5),
                                       transforms.RandomApply(transforms=[gaussian_noise(0, 0.35)], p=0.3),
                                       transforms.RandomApply(transforms=[salt_pepper_noise(0.35)], p=0.3),
                                       transforms.RandomRotation((0, 10)),
                                       transforms.RandomGrayscale(0.1)])

train_dataset = HDF5Dataset(data_hdf5_root, name = "train", transform = train_transforms)

In [ ]:
batch_size = 128
n_epochs = 10

model = torchvision.models.resnet18(pretrained=True)
input_num_fc = model.fc.in_features
model.fc = nn.Linear(input_num_fc, 29, bias=True)
model = model.to(device=device)

optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum=0.9)

loss_fn = nn.CrossEntropyLoss()

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
model_path = os.path.join(path, "data/asl/aug_models/ResNet18_aug3.pt")
res = train_loop(n_epochs, train_loader, val_loader, model, loss_fn, optimizer, model_path)

#### The 4th augmentation

In [ ]:
train_transforms = transforms.Compose([transforms.ToTensor(),
                                       transforms.Normalize((0.5190, 0.4993, 0.5141),
                                                         (0.2280, 0.2555, 0.2637)),
                                       transforms.Resize(224),
                                       transforms.RandomHorizontalFlip(p=0.5),
                                       transforms.RandomApply(transforms=[gaussian_noise(0.1, 0.2)], p=0.3),
                                       transforms.RandomApply(transforms=[salt_pepper_noise(0.2)], p=0.3),
                                       transforms.RandomRotation((0, 5))])

train_dataset = HDF5Dataset(data_hdf5_root, name = "train", transform = train_transforms)

In [ ]:
batch_size = 128
n_epochs = 10

model = torchvision.models.resnet18(pretrained=True)
input_num_fc = model.fc.in_features
model.fc = nn.Linear(input_num_fc, 29, bias=True)
model = model.to(device=device)

optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum=0.9)

loss_fn = nn.CrossEntropyLoss()

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
model_path = os.path.join(path, "data/asl/aug_models/ResNet18_aug4.pt")
res = train_loop(n_epochs, train_loader, val_loader, model, loss_fn, optimizer, model_path)

#### Training without augmentation

In [ ]:
train_dataset = HDF5Dataset(data_hdf5_root, name = "train", transform = test_transforms)
# validation dataset is same

In [ ]:
batch_size = 128
n_epochs = 10

model = torchvision.models.resnet18(pretrained=True)
input_num_fc = model.fc.in_features
model.fc = nn.Linear(input_num_fc, 29, bias=True)
model = model.to(device=device)

optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum=0.9)

loss_fn = nn.CrossEntropyLoss()

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

model_path = os.path.join(path, "data/asl/aug_models/ResNet18_noaug.pt")
res = train_loop(n_epochs, train_loader, val_loader, model, loss_fn, optimizer, model_path)

#### Test!

In [8]:
def test_loop(test_loader, model, loss_fn):
    loss_total = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        model.eval()
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, predicted = torch.max(outputs, dim=1)
            total += labels.shape[0]
            correct += int((predicted == labels).sum())
            loss = loss_fn(outputs, labels)
            loss_total += loss.item()

        acc_test = 100*correct/total
    print('Total loss {:.4f}, Total accuracy {:2.3f}%'
            .format(loss_total/len(test_loader), acc_test))
    return acc_test, correct, predicted, total

def accuracy_test(test_loader, test2_loader, model, loss_fn):
  print("Test Data:")
  acc_test, cn, predict, total_num= test_loop(test_loader, model, loss_fn)
  print(f"accuracy : {acc_test :2.3f}, correct number : {cn}/{total_num}")
  print("Test2 Data:")
  acc_test2, cn2, predict2, total_num2= test_loop(test2_loader, model, loss_fn)
  print(f"accuracy : {acc_test2 :2.3f}, correct number : {cn2}/{total_num2}")

  return (acc_test, cn, acc_test2, cn2)

In [9]:
test_dataset = HDF5Dataset(data_hdf5_root, name="test", transform = test_transforms)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)
test2_dataset = HDF5Dataset(data_hdf5_root, name="test2", transform = test_transforms)
test2_loader = DataLoader(test2_dataset, batch_size=len(test2_dataset), shuffle=False)

loss_fn = nn.CrossEntropyLoss()

In [12]:
models_root = os.path.join(path, "data/asl/aug_models/")
for f in glob.glob(os.path.join(models_root, '*.pt')):
    print(f.split("/")[-1].split(".")[0], ":")
    model = torch.jit.load(f)
    model = model.to(device)
    _ = accuracy_test(test_loader, test2_loader, model, loss_fn)
    print("---------------------------------")

ResNet18_aug1 :
Test Data:
Total loss 0.0053, Total accuracy 100.000%
accuracy : 100.000, correct number : 29/29
Test2 Data:
Total loss 3.1349, Total accuracy 24.016%
accuracy : 24.016, correct number : 415/1728
---------------------------------
ResNet18_noaug :
Test Data:
Total loss 0.0069, Total accuracy 100.000%
accuracy : 100.000, correct number : 29/29
Test2 Data:
Total loss 3.3366, Total accuracy 15.799%
accuracy : 15.799, correct number : 273/1728
---------------------------------
ResNet18_aug3 :
Test Data:
Total loss 0.0036, Total accuracy 100.000%
accuracy : 100.000, correct number : 29/29
Test2 Data:
Total loss 3.3422, Total accuracy 23.553%
accuracy : 23.553, correct number : 407/1728
---------------------------------
ResNet18_aug4 :
Test Data:
Total loss 0.0057, Total accuracy 100.000%
accuracy : 100.000, correct number : 29/29
Test2 Data:
Total loss 3.0620, Total accuracy 23.553%
accuracy : 23.553, correct number : 407/1728
---------------------------------
ResNet18_aug2 :